In [1]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tabulate import tabulate
import yaml

# Reading in data

In [2]:
#values for csv importing
csv_opts = {'sep': '|',
           'quotechar': '"',
           #'compression': 'gzip',
           'encoding': 'utf-8'}

In [6]:
with open('../../data/arrest_dtypes.yaml', 'r') as yamlfile:
        arrest_dtypes = yaml.load(yamlfile, Loader=yaml.FullLoader)
#reading in CSV file
arrests = pd.read_csv('../../data/arrests.csv', **csv_opts, dtype=arrest_dtypes)

with open('../../data/encounter_dtypes.yaml', 'r') as yamlfile:
        arrest_dtypes = yaml.load(yamlfile, Loader=yaml.FullLoader)
#reading in CSV file
encounters = pd.read_csv('../../data/encounters.csv', **csv_opts, dtype=arrest_dtypes)

with open('../../data/removal_dtypes.yaml', 'r') as yamlfile:
        arrest_dtypes = yaml.load(yamlfile, Loader=yaml.FullLoader)
#reading in CSV file
removals = pd.read_csv('../../data/removals.csv', **csv_opts, dtype=arrest_dtypes)

#reading in CSV file
facilities = pd.read_csv('../../data/ice-facilities.csv')

In [7]:
#putting event date into m/d/y format
arrests['apprehension_date'] = pd.to_datetime(
    arrests['apprehension_date'], format='%m/%d/%Y')

#putting event date into m/d/y format
encounters['event_date'] = pd.to_datetime(encounters['event_date'], format='%m/%d/%Y')

#putting date variables into m/d/y format
removals['apprehension_date'] = pd.to_datetime(removals['apprehension_date'], 
                                               format='%m/%d/%Y', errors='coerce')
removals['departed_date'] = pd.to_datetime(removals['departed_date'], format='%m/%d/%Y')
removals['removal_date'] = pd.to_datetime(removals['removal_date'], format='%m/%d/%Y')

In [9]:
# adding year and month columns
arrests["year"] = arrests["apprehension_date"].dt.strftime('%Y')
arrests["month"] = arrests["apprehension_date"].dt.strftime('%m')

arrests.set_index('apprehension_date').groupby(
    pd.Grouper(freq='AS-OCT'))['id'].count()

arrests.set_index('apprehension_date').groupby(
    pd.Grouper(freq='MS'))['id'].count()

## Plot 2 idea
- dropdown to select aor
- then see encounters, removals, and arrests all on same plot

In [8]:
import plotly.express as px

In [17]:
fy = ['2015-10-01', '2016-10-01', '2017-10-01', '2018-10-01']

In [21]:
aor = np.unique(facilities['aor']).tolist()

## Arrests by FY plot

In [23]:
# creating subset of arrests by AOR over time
arrests_by_fy = arrests[["aor", "apprehension_date", "id"]]
arrests_by_fy = arrests_by_fy.groupby(
    ["aor", "apprehension_date"], as_index=False)['id'].count()

# pivot table so dates are rows and AORs are columns
arrests_by_fy = arrests_by_fy.pivot(
    index='apprehension_date', columns='aor', values='id')

# grouping by fiscal year
arrests_by_fy = arrests_by_fy.groupby(
    pd.Grouper(freq='AS-OCT'), dropna=False).sum()

In [24]:
fig = px.line(arrests_by_fy, x=fy, 
              y=aor, 
              title = "Arrests in AOR per FY",
              labels=dict(x="Fiscal Year", y="Number of Arrests"))
fig.update_xaxes(title="Fiscal Year", nticks = 4)
fig.update_yaxes(title="Number of Arrests")
fig.update_layout(legend_title_text='AOR')
fig.show()

## Removals by fy

In [25]:
# create subset that counts the removals in each AOR over time
removals_by_fy = removals[["aor", "removal_date", "id"]]
removals_by_fy = removals_by_fy.groupby(["aor", "removal_date"], as_index=False)['id'].count()

# pivot table so dates are in rows and AOR is in columns
removals_by_fy = removals_by_fy.pivot(index='removal_date', columns='aor', values='id')

# group by fiscal year
removals_by_fy = removals_by_fy.groupby(pd.Grouper(freq='AS-OCT')).sum()

# # create plot of removals over time
# removals_by_fy.plot().legend(loc='center left',bbox_to_anchor=(1.1, 0.5))

# # create transposed table of removals in each AOR by fiscal year
# removals_by_fy.T.sort_values(by=["2018-10-01 00:00:00"], ascending=False).style.format("{:,.0f}")

In [26]:
fig = px.line(removals_by_fy, x=fy, 
              y=aor, 
              title = "Removals in AOR per FY",
              labels=dict(x="Fiscal Year", y="Number of Removals"))
fig.update_xaxes(title="Fiscal Year", nticks = 4)
fig.update_yaxes(title="Number of Removals")
fig.update_layout(legend_title_text='AOR')
fig.show()

## Encounters by FY

In [27]:
# create subset of just aor, date, and id
encounters_by_fy = encounters[["aor", "event_date", "id"]]

# group subset by aor and date, then count ids
encounters_by_fy = encounters_by_fy.groupby(["aor", "event_date"], 
                                            as_index=False, dropna=False)['id'].count()

# pivot subset so that index is the event date and columns are aor (allows for easy Grouper in next line of code)
encounters_by_fy = encounters_by_fy.pivot(index='event_date', 
                                          columns='aor', values='id')

# group subset by fiscal year and total all the values 
encounters_by_fy = encounters_by_fy.groupby(pd.Grouper(freq='AS-OCT'), dropna=False).sum()

In [28]:
fig = px.line(encounters_by_fy, x=fy, 
              y=aor, 
              title = "Encounters in AOR per FY",
              labels=dict(x="Fiscal Year", y="Number of Encounters"))
fig.update_xaxes(title="Fiscal Year", nticks = 4)
fig.update_yaxes(title="Number of Encounters")
fig.update_layout(legend_title_text='AOR')
fig.show()

# combined per fy for given aor

In [72]:
date = encounters_by_fy[aor_].index.tolist()
enc = encounters_by_fy[aor_].tolist()
rem = removals_by_fy[aor_].tolist()
arr = arrests_by_fy[aor_].tolist()

In [75]:
columns_ = ['date', 'encounters', 'removals', 'arrests']



In [73]:
temp = pd.DataFrame(data = data_, columns = columns_)



In [74]:
temp

,date,encounters,removals,arrests
0,2015-10-01,33227,5770,8866
1,2016-10-01,28263,12571,13551
2,2017-10-01,28876,13727,15189
3,2018-10-01,25275,13061,12268


In [83]:
fig = px.line(temp, x=temp['date'], 
              y=[temp['encounters'], temp['removals'], temp['arrests']],
              title = "Encounters, Removals, and Arrests in given AOR per FY",
              labels=dict(x="Fiscal Year", y="Number of Encounters"))
fig.update_xaxes(title="Fiscal Year", nticks = 4)
fig.update_yaxes(title="Number of Encounters, Removals, or Arrests")
fig.update_layout(legend_title_text='Encounters, Removals, and Arrests key')
fig.show()